In [36]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer

/Users/zhiji/anaconda/envs/Fintech/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/zhiji/anaconda/envs/Fintech/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# 1. load dataframe from cvs files

In [3]:
path = "/Users/zhiji/Desktop/实习/ccts/黑产攻击识别/"
train = pd.read_csv(path + "atec_anti_fraud_train.csv")
test = pd.read_csv(path + "atec_anti_fraud_test_a.csv")

# 2. pre-process the data

In [86]:
#there is totally 297 features

train_labeled_raw = train[train.label != -1]
# drop all nan features
train_labeled = train_labeled_raw.fillna(0)
print(len(train_labeled_raw))
features = []
for c in train_labeled.columns:
    if c[0] == 'f':
        features.append(c)
x = train_labeled[features]
y = train_labeled.label

990006


# 3. Fit machine learning methods

In [ ]:
gammas, Cs = [], []
for a in np.r_[-15: 4][::2]:
    gamma = pow(float(2), a)
    gammas.append(gamma)
for b in np.r_[-5: 16][::2]:
    c = pow(float(2), b)
    Cs.append(c)
# Set the parameters by cross-validation  
tuned_parameters = {'kernel': ['rbf'], 'gamma': gammas,  'C': Cs}
stat_svm = get_stat_SVM(x, y, tuned_parameters, split = 0.8)

In [31]:
def svm_distribution(label, classnum):
  if (classnum == 2):
    z, t = ZeroTwoFrequency(label)
    return 'Up：' + str(z) + ' Down：' + str(t)
  if (classnum == 3):
    z, o, t = ZeroOneTwoFrequency(label)
    return 'Up：' + str(z) + ' Flat: ' + str(o) + ' Down：' + str(t)
def get_stat_SVM(X, y, tuned_parameters, split = 0.8):
    d_test, d_train = [], []
    a_train = []
    a_test =  []
    classnum = 2
    #get cleaned technical indicators
#     X, y = get_dataframe(name, iwl, fh, start_date, end_date,freq, classnum = classnum, method = 'SVM')
    edge = np.int(split*len(y))
    X_train, X_test, y_train, y_test = X[:edge], X[edge:],y[:edge],y[edge:]
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_nomalized = scaler.transform(X_train)
    X_test_nomalized = scaler.transform(X_test)

    clf = GridSearchCV(SVC(), tuned_parameters, cv = 5,
                  scoring = 'accuracy')
    clf.fit(X_train_nomalized, y_train)
    prediction_train = clf.predict(X_train_nomalized)
    prediction_test = clf.predict(X_test_nomalized)
    print("2 classes prediction: ", prediction_train, prediction_test)
    a_train.append(accuracy_score(y_train, prediction_train))
    a_test.append(accuracy_score(y_test, prediction_test))
#     z_train, t_train = ZeroTwoFrequency(y_train)
#     z_test, t_test = ZeroTwoFrequency(y_test)

    d_train.append(svm_distribution(y_train, classnum))
    d_test.append(svm_distribution(y_test, classnum))

    return pd.DataFrame({
      '训练分布':d_train,
      '测试分布':d_test,
      '训练精度SVM': a_train,
      '测试精度SVM': a_test
    },index = idx)

In [ ]:
def ann(inputs,bs, epochs, neuron_num,verbose = False ):
    layer_num = len(neuron_num)
    nx = neuron_num[0]
    ny = neuron_num[layer_num - 1]
    weights, biases = {}, {}
    x = tf.placeholder('float32',[None, nx])
    y = tf.placeholder('float32',[None, ny])
    layer_l = x
    for l in range(layer_num - 2):
        input_num = neuron_num[l]
        hidden_num = neuron_num[l + 1]
        weights['h' + str(l+1)] = tf.Variable(tf.random_normal([input_num, hidden_num]))
        biases['b' + str(l+1)] = tf.Variable(tf.random_normal([hidden_num]))

        layer_l = tf.nn.relu(tf.add(tf.matmul(layer_l, weights['h' + str(l+1)]), biases['b' + str(l+1)]))

    weights['out'] = tf.Variable(tf.random_normal([hidden_num, ny]))
    biases['out'] = tf.Variable(tf.random_normal([ny]))
    prediction = tf.nn.softmax(tf.matmul(layer_l, weights['out']) + biases['out'])
    tf.add_to_collection('pred', y)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        losses, epoches = [],[]
        for epoch in range(epochs):
            epoch_loss=0
            i=0
            while i < len(X_nomalized):
                start = i
                end = i + bs
                batch_x = np.array(X_nomalized[start:end])
                batch_y = np.array(Y[start:end])

                _,c = sess.run([optimizer,cost] , feed_dict = {x: batch_x , y : batch_y})
                if verbose:
                    epoch_loss+= c
                i+= bs
            if (verbose) :
                if (epoch % 1000 == 0):
                    print("Epoch",epoch , 'completed out of ' ,epochs, ' loss: ', epoch_loss )
                losses.append(epoch_loss)
                epoches.append(epoch)

In [14]:
def save_model_ANN(iwl, fh, name, bs, epochs, neuron_num, start_date, end_date,freq, verbose = False):
  classnum = neuron_num[-1]
  X, Y = get_dataframe(name, iwl, fh, start_date, end_date, freq, classnum = classnum, method = 'ANN')
  scaler = preprocessing.StandardScaler().fit(X)
  X_nomalized = scaler.transform(X)

  layer_num = len(neuron_num)
  nx = neuron_num[0]
  ny = neuron_num[layer_num - 1]
  weights, biases = {}, {}
  #get nomalized train and test set


  x = tf.placeholder('float32',[None, nx])
  y = tf.placeholder('float32',[None, ny])
  layer_l = x
  for l in range(layer_num - 2):
    input_num = neuron_num[l]
    hidden_num = neuron_num[l + 1]
    weights['h' + str(l+1)] = tf.Variable(tf.random_normal([input_num, hidden_num]))
    biases['b' + str(l+1)] = tf.Variable(tf.random_normal([hidden_num]))

    layer_l = tf.nn.relu(tf.add(tf.matmul(layer_l, weights['h' + str(l+1)]), biases['b' + str(l+1)]))

  weights['out'] = tf.Variable(tf.random_normal([hidden_num, ny]))
  biases['out'] = tf.Variable(tf.random_normal([ny]))
  prediction = tf.nn.softmax(tf.matmul(layer_l, weights['out']) + biases['out'])
  tf.add_to_collection('pred', y)

  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
  optimizer = tf.train.AdamOptimizer().minimize(cost)
  saver = tf.train.Saver()

  with tf.Session() as sess:
      sess.run(tf.initialize_all_variables())
      losses, epoches = [],[]
      for epoch in range(epochs):
          epoch_loss=0
          i=0
          while i < len(X_nomalized):
              start = i
              end = i + bs
              batch_x = np.array(X_nomalized[start:end])
              batch_y = np.array(Y[start:end])

              _,c = sess.run([optimizer,cost] , feed_dict = {x: batch_x , y : batch_y})
              if verbose:
                  epoch_loss+= c
              i+= bs
          if (verbose) :
                if (epoch % 1000 == 0):
                  print("Epoch",epoch , 'completed out of ' ,epochs, ' loss: ', epoch_loss )
                losses.append(epoch_loss)
                epoches.append(epoch)
      path = "./"
      model_name = "ann" + str(classnum) +".ckpt"
      print("model name is : " + model_name)
      saver.save(sess, path + model_name)

Index(['id', 'date', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8',
       ...
       'f288', 'f289', 'f290', 'f291', 'f292', 'f293', 'f294', 'f295', 'f296',
       'f297'],
      dtype='object', length=299)